In [7]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  
import pandas as pd
import numpy as np
np.random.seed(0)

# from aif360.datasets import GermanDataset
from aif360.datasets import StructuredDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

In [8]:
df = pd.read_csv("adultData2.csv")

In [9]:
df.head()

,Age,Workclass,fnlwgt,Education,Education-num,Marrital-Status,Occupation,Relationship,Race,Sex,Capital-Gain,Capital-Loss,Hours-Per-Week,Native-Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Protected Class = Sex

##### Priveleged Class = Male
##### Unprivileged Class = Female

In [10]:
# outcome could be a np array i think
def dataset_wrapper(outcome, protected, unprivileged_groups, privileged_groups,
                          favorable_label, unfavorable_label):
    """ A wraper function to create aif360 dataset from outcome and protected in numpy array format.
    """
    df = pd.DataFrame(data=outcome,
                      columns=['outcome'])
    df['race'] = protected
    
    dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                       unfavorable_label=unfavorable_label,
                                       df=df,
                                       label_names=['outcome'],
                                       protected_attribute_names=['race'],
                                       unprivileged_protected_attributes=unprivileged_groups)
    return dataset

In [11]:
dataset = StructuredDataset(df, label_names=['outcome'], protected_attribute_names=["Sex"])
    
#     favorable_label=1.0,
#                                        unfavorable_label=0.0,
#                                        df=df,
#                                        label_names=['outcome'],
#                                        protected_attribute_names=['Sex'])

ValueError: could not convert string to float: ' >50K'


ValueError: DataFrame values must be numerical.

In [12]:
dataset_orig = dataset_wrapper(protected_attribute_names=['Sex'],           # this dataset also contains protected
                                                                          # attribute for "sex" which we do not
                                                                          # consider in this evaluation
                             privileged_classes=[lambda x: x == "Male"],      # age >=25 is considered privileged
                             features_to_drop=['Age', 'Race', 'Marital-Status', 'Native-Country', 'Relationship']) # ignore sex-related attributes

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'Sex': "Male"}]
unprivileged_groups = [{'Sex': "Female"}]

TypeError: dataset_wrapper() got an unexpected keyword argument 'protected_attribute_names'